In [1]:
import os
import json

MAX_THEOREMS = 10

class GetTheorem:
    def __init__(self):
        self.tasks = []
        file_path = os.path.join("dataprep", "1hop_trueontology_formalized.json")
        with open(file_path, "r") as file:
            json_theorems = json.load(file)
            for i in range(1, MAX_THEOREMS + 1):
                self.tasks.append(json_theorems['example' + str(i)]['test_example']['formalized'])

    def get_theorem(self, theorem = 1):
        return self.tasks[theorem-1]

    

In [2]:
import gymnasium as gym
import logic_gym
import graphviz

In [3]:
get_theorem = GetTheorem()

theorem = get_theorem.get_theorem(1)


In [4]:
dot = graphviz.Digraph()

# Adding nodes
for level in range(1, 6):
    dot.node(str(level), f"Node {level}")

# Adding edges
# dot.edges(['12', '23', '34', '45', '51'])
dot.edge("1", "2", "12")
dot.edge("2", "3")
dot.edge("3", "4")
dot.edge("4", "5")
dot.edge("5", "1")

# Render the graph
dot.render("sample_graph", format="png", view=False)

'sample_graph.png'

In [5]:
class Node:
    def __init__(
        self,
        index,
        level,
        observation_before,
        action,
        observation_after,
        is_done,
        terminated,
        truncated,
        proof,
        parent=None,
        children= None,
        reward=0,
        logic_gym_state=None,
    ):
        self.index = index
        self.level = level
        self.observation_before = observation_before
        self.action = action
        self.observation_after = observation_after
        self.is_done = is_done
        self.terminated = terminated
        self.truncated = truncated
        self.proof = proof
        self.parent = parent
        self.children = children if children is not None else []
        self.reward = reward
        self.logic_gym_state=logic_gym_state


    def add_child(self, child_node):
        self.children.append(child_node)
        child_node.parent = self
    def __str__(self) -> str:
        return f"Node {self.index} at level {self.level} with action {self.action} and reward {self.reward}"

In [6]:
def prove_theorem_by_search(env, depth = 3, exit_on_proof = False):
    root = Node(
        index=0,
        level=0,
        observation_before=None,
        action=None,
        observation_after=env.reset(),
        is_done=False,
        terminated=False,
        truncated=False,
        proof=None,
    )
    root.logic_gym_state = env.unwrapped.get_state()
    nodes = [root]
    nodes_by_level = {level: [] for level in range(depth + 1)}
    nodes_by_level[0] = [root]
    
    index = 1
    
    
    for level in range(depth-1):
        for node in nodes_by_level[level]:
            for action in env.unwrapped.get_all_actions():
                # print(node.index, node.level, action)
                # print(node.logic_gym_state)
                env.unwrapped.set_state(node.logic_gym_state)
                # print(node.index, node.level)
                # print(node.logic_gym_state["state"])
                # print("--------------------------------------")
                observation, reward, terminated,truncated, info = env.unwrapped.step(action)
                if info["bad_action"]: # check if the action is valid
                    continue
                
                is_done = terminated or truncated
                state = env.unwrapped.get_state()   
                child = Node(
                    index=index,
                    level=node.level + 1,
                    observation_before=node.observation_after,
                    action=action,
                    observation_after=observation,
                    is_done=is_done,
                    terminated=terminated,
                    truncated=truncated,
                    proof=state["state"],
                    reward=reward,
                    logic_gym_state=state,
                    parent=node,
                )
                if terminated:
                    print("proof found")
                    print(child.proof)
                    
                index += 1
                node.add_child(child)
                nodes_by_level[level+1].append(child)
                if is_done and exit_on_proof:
                    break
            if is_done and exit_on_proof:
                break
        if is_done and exit_on_proof:
            break
    return root, nodes_by_level
    

    
    

In [7]:
env = gym.make("logic_gym/LogicGym-v0")

env.set_task(theorem)


root, nodes_by_level = prove_theorem_by_search(env, depth=3, exit_on_proof=False)

print(root)

/home/akash/anaconda3/envs/fol/lib/python3.11/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.set_task to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.set_task` for environment variables or `env.get_wrapper_attr('set_task')` that will search the reminding wrappers.
  logger.warn(


proof found
Ax.(Cat(x) -> Feline(x))                                      (0)  Given
Ax.(Feline(x) -> Carnivore(x))                                (1)  Given
Ax.(Carnivore(x) -> ~Herbivorous(x))                          (2)  Given
Ax.(Snake(x) -> ~WarmBlooded(x))                              (3)  Given
Ax.(Carnivore(x) -> Mammal(x))                                (4)  Given
Ax.(Mammal(x) -> WarmBlooded(x))                              (5)  Given
Ax.(Mammal(x) -> Vertebrate(x))                               (6)  Given
Ax.(Vertebrate(x) -> Animal(x))                               (7)  Given
Ax.(Animal(x) -> Multicellular(x))                            (8)  Given
Mammal(Sally)                                                 (9)  Given
WarmBlooded(Sally)                                           (10)  Goal
Mammal(Sally) -> WarmBlooded(Sally)                          (11)  A-Elimination (5), with Sally
WarmBlooded(Sally)                                           (12)  Implication-Eliminatio

In [8]:
def BFS(root, dot):
    queue = [root]
    while queue:
        node = queue.pop(0)
        dot.node(str(node.index), f"Node {node.index}, terminal: {node.terminated}")
        if node.parent is not None:
            dot.edge(str(node.parent.index), str(node.index), f"Action: {node.action}")
        print(node)
        queue.extend(node.children)


In [9]:
dot = graphviz.Digraph()
BFS(root, dot)
dot.render("sample_graph", format="png", view=False)

Node 0 at level 0 with action None and reward 0
Node 1 at level 1 with action [0, 7, 0, 0] and reward 0
Node 2 at level 1 with action [0, 2, 0, 0] and reward 0
Node 3 at level 1 with action [0, 8, 0, 0] and reward 0
Node 4 at level 1 with action [0, 6, 0, 0] and reward 0
Node 5 at level 1 with action [0, 1, 0, 0] and reward 0
Node 6 at level 1 with action [0, 5, 0, 0] and reward 0
Node 7 at level 1 with action [0, 4, 0, 0] and reward 0
Node 8 at level 1 with action [0, 3, 0, 0] and reward 0
Node 9 at level 1 with action [0, 0, 0, 0] and reward 0
Node 10 at level 2 with action [0, 3, 0, 0] and reward 0
Node 11 at level 2 with action [0, 7, 0, 0] and reward 0
Node 12 at level 2 with action [0, 0, 0, 0] and reward 0
Node 13 at level 2 with action [0, 6, 0, 0] and reward 0
Node 14 at level 2 with action [0, 1, 0, 0] and reward 0
Node 15 at level 2 with action [0, 8, 0, 0] and reward 0
Node 16 at level 2 with action [0, 4, 0, 0] and reward 0
Node 17 at level 2 with action [0, 2, 0, 0] and r

'sample_graph.png'

In [10]:
len(root.children)

9